In [1]:
%pwd


'/Users/ujjwalraj/Desktop/Medical Chatbot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/ujjwalraj/Desktop'

In [5]:
#to go inside the directory
%cd Medical Chatbot



/Users/ujjwalraj/Desktop/Medical Chatbot


In [6]:
%pwd

'/Users/ujjwalraj/Desktop/Medical Chatbot'

In [14]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [19]:
%pwd

'/Users/ujjwalraj/Desktop/Medical Chatbot'

In [20]:
extracted_data=load_pdf_file(data='data/')


In [21]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [22]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

In [26]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [27]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
HUGGINGFACE_ACCESS_TOKEN=os.environ.get('HUGGINGFACE_ACCESS_TOKEN')



In [30]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-ir892vd.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ['HUGGINGFACE_ACCESS_TOKEN']=HUGGINGFACE_ACCESS_TOKEN

In [32]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [33]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [43]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='d49a9502-28d6-4e6d-851d-644b485e5fab', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a574a11e-3c31-48bf-9d68-62cd106482e3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0

In [ ]:
from langchain_community.llms import HuggingFaceHub  
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os

# ✅ Set Hugging Face API Token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = 'your_huggingface_api_token' 

# ✅ Load Hugging Face LLM
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={"temperature": 0.4, "max_new_tokens": 500},
)

# ✅ Reinitialize Pinecone retriever (ensure index_name and embeddings are defined)
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
index_name = "medicalbot"

# Connect to existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# ✅ Initialize the retriever properly
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# ✅ Define Prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# ✅ Create Chains
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# ✅ Invoke the chain
response = rag_chain.invoke({"input": "What is Acromegaly and Gigantism?"})
print(response["answer"])

/Users/ujjwalraj/Desktop/Medical Chatbot/venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Whitehouse Station, NJ: Merck Research Laboratories,
1997.
Larsen, D. E., ed. Mayo Clinic Family Health Book.New York:
William Morrow and Co., Inc., 1996.
John T. Lohr, PhD
Acromegaly and gigantism
Definition
Acromegaly is a disorder in which the abnormal
release of a particular chemical from the pituitary gland
in the brain causes increased growth in bone and soft tis-
sue, as well as a variety of other disturbances throughout
the body. This chemical released from the pituitary gland

mone Excess: Acromegaly and Gigantism.” In Harrison’s
Principles of Internal Medicine, ed. Anthony S. Fauci, et
al. New York: McGraw-Hill, 1997.
Jameson, J. Larry. “Growth Hormone Excess: Acromegaly and
Gigantism.” In Cecil Textbook of Medicine, ed. J. Claude
Bennett

In [54]:
response = rag_chain.invoke({"input": "whale blood test?"})
print(response["answer"])

/Users/ujjwalraj/Desktop/Medical Chatbot/venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

the same ABO and Rh type as the rcipient is selected. In
a test tube, serum from the patient is mixed with red
blood cells from the donor. If clumping occurs, the blood
is not compatible; if clumping does not occur, the blood
is compatible. If an unexpected antibody is found in
either the patient or the donor, the blood bank does fur-
ther testing to make sure the blood is compatible.
In an emergency, when there is not enough time for
blood typing and crossmatching, O red blood cells may

conditions.
• Chorionic villus sampling (CVS). This test can be done
as early as the ninth week of pregnancy to identify
chromosome disorders and some genetic conditions. A
thin needle is inserted through the abdomen or a slim
tube is inserted through the vagina t

In [50]:
import requests

headers = {"Authorization": f"Bearer {os.environ['HUGGINGFACEHUB_API_TOKEN']}"}
response = requests.get("https://huggingface.co/api/whoami", headers=headers)

print(response.json())  # ✅ Should return your Hugging Face user details if valid

{'error': 'Invalid credentials in Authorization header'}
